# Nilearn: From fMRI to scikit-learn

## Vectorizing is really useful...

Scikit-learn has a recurring pattern in its API. Whether you're fitting a regressor, a classifier, or a feature transformation object, they almost all take data as an `X` and (sometimes) `y` array.

However, in neuroimaging (and many other fields) our data doesn't naturally come in this form.

* sklearn shapes: (samples x features)
* nipy shapes: (x, y, z, time/conditions/etc)

The process of converting your Nifti-shaped files into a scikit-learn format is often called **vectorizing**. Effectively, this means going from:

**your data shape -> (samples, features)**

This notebook shows you how to do this in the context of `nilearn`.

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pylab as plt
import os.path as op
%matplotlib inline

In [ ]:
from nilearn import datasets
from nilearn import plotting
from nilearn import image

# By default the 2nd subject will be fetched
haxby_dataset = datasets.fetch_haxby()
fmri_filename = haxby_dataset.func[0]

# print basic information on the dataset
print('First subject functional nifti images (4D) are at: %s' %
      fmri_filename)  # 4D data

## Preparing the fMRI volume for model fitting
Machine learning generally follows this pattern:

1. Data preprocessing
2. feature extraction/engineering
3. model fitting
4. model validation
5. model inspection

This covers a part of point 2.

### Extracting a subset of voxels
We can easily load volume data into python using `nibabel`. However, we generally don't want the full volume, but instead prefer to use a subset of voxels based on some preferred region of the brain (e.g. the cortical surface).

Effectively, we wish to do two things:

1. Extract a subset of voxels from the nifty file (masking)
2. Reshape these voxels so that they can be used to fit a model (vectorizing)

`nilearn` allows us to do this easily, turning the 4-D matrix of into an array of shape (n_samples, n_features). This allows us to use scikit-learn to do machine learning.

### Visualizing the mask
First, remember that masks are simply a 3D array in the same space as our MRI data. They are basically boolean values that say whether or not to keep each voxel. 

We'll load and plot it below. The mask is a mask of the Ventral Temporal streaming area coming from the Haxby study:

In [ ]:
mask_filename = haxby_dataset.mask_vt[0]
plotting.plot_roi(mask_filename, bg_img=haxby_dataset.anat[0],
                  cmap='Paired')

Notice that only a few of the voxels are being shown here. These are the voxels that the mask has extracted. Now we use the NiftiMasker.

### Masking / vectorizing our data
We will use the `nilearn.input_data.NiftiMasker` to extract the
fMRI data on a mask and convert it to data series.

We first create a masker, giving it the options that we care
about. Here we also use standardizing of the data, as it is often important
for decoding. This scales the data so that its mean / variance is more consistent across voxels.

In [ ]:
from nilearn.input_data import NiftiMasker
masker = NiftiMasker(mask_img=mask_filename, standardize=True)
masker

**A note on transformers**

The `masker` is an object that can perform data masking for us. It follows a very similar API to `scikit-learn`. Objects that transform other objects are called `transformers`, and generally have a `transform` method on top of the `fit` method that we covered earlier.

We'll `fit` the masker on our mask, then use it to vectorize our data.

In [ ]:
# Fitting the transformer initializes it to operate on new data
masker.fit(fmri_filename)

In [ ]:
# Now we'll transform our fMRI data
fmri_masked = masker.transform(fmri_filename)

The variable "fmri_masked" is a numpy array. It is 2-D.

In [ ]:
print(fmri_masked)

Its shape corresponds to the number of time-points x the number of
voxels in the mask. Note that this is much fewer than the total number of voxels in the nifty image.

In [ ]:
print(fmri_masked.shape)

### De-vectorizing back to a nifti image
After we've done our analysis using the vectorized data, we then want to reshape it back to `i,j,k` space. 

Using our `NiftiMasker`, we can turn the dta back into a Nifti image, in essence, "inverting"
what the NiftiMasker has done.

For this, we can call `inverse_transform` on the NiftiMasker:

In [ ]:
nifti_img = masker.inverse_transform(fmri_masked)
print(nifti_img)

In [ ]:
plotting.plot_stat_map(image.index_img(nifti_img, 2))

Note that maskers will work for different kinds of inputs to `inverse_transform`...

In [ ]:
# Note that this will work fine
nifti_img = masker.inverse_transform(fmri_masked[:10])
print(nifti_img.shape)

In [ ]:
# Or even this...
nifti_img = masker.inverse_transform(fmri_masked[0])
print(nifti_img.shape)

In [ ]:
# But changing the number of features will break it...
nifti_img = masker.inverse_transform(fmri_masked[:, :-1])

### Vectorizing (by itself) removes information

Vectorizing is often the final step of the processing chain before we actually fit a model on our data. It often leaves the data in a state with _minimal_ knowledge about the relationships between features. Make sure that you keep track of what each feature means (or use a tool like the Nifti masker) to know how to interpret your model results. 